In [ ]:
import numpy as np
import re
import nltk
from sklearn import datasets
nltk.download('stopwords')
from nltk.corpus import stopwords
import pickle
import email
import os
from bs4 import BeautifulSoup 
from models.config.cleaning_methods import decode_mime_stuff
import datetime as dt
from models.config.filter_methods import testing_platform_filters
from models.config.CONSTS import CONSTS

In [ ]:
'''
Stripped down version of the platform parsing methods dict.
Creating functions to streamline data creation.
'''


platform_identification = {
    "linkedin.com":"LinkedIn",
    "untapped.gg":"Untapped",
    "ziprecruiter.com":"ZipRecruiter",
    "squarepeg.com":"SquarePeg",
}

platform_filters = {
    'LinkedIn' : {
        'email': {
            'match' : lambda x : "X-LinkedIn-Template" in x['headers'].keys(),
            'scan' : {
                'date'        : lambda x : dt.datetime.strptime(re.sub(r"\s\(\w{3}\)","", x['headers']['Received'].split(';')[1].split(",")[1].strip()), '%d %b %Y %H:%M:%S %z').strftime(CONSTS['date_format']), #standard to gmail format; gets datetime of email sent
                'update_type' : lambda x : re.search(r'jobs?_appli\w{4,6}_([a-z]+)', x['headers']['X-LinkedIn-Template']).group(1), #type of email update
                'subject'     : lambda x : decode_mime_stuff(x['headers']['Subject']).replace(",","").replace(".","").replace("&", "and").split(":")[0],
                # 'title' : lambda x : re.search(x,r''), #title of job posting
                # 'company' : lambda x : re.search(x,r''), #company name
            },
            'applied' : {
                'PostID'   : lambda x : re.search(r"(\d{10})",x['body'].find("a", href=re.compile(r"https://www.linkedin.com/comm/jobs/view/"))['href']).group(1),
                'url'      : lambda x : 'https://www.linkedin.com/jobs/view/{}/'.format(re.search(r"(\d{10})",x['body'].find("a", href=re.compile(r"https://www.linkedin.com/comm/jobs/view/"))['href']).group(1)),
                'position' : lambda x : x['subject'].split(" at ")[0],
                'company'  : lambda x : x['subject'].split(" at ")[1],
            },
            'viewed' : {
                'PostID'   : lambda x : re.search(re.compile(r'jobPostingId%3D(\d{10})%26pivotType%3Dsim'), x['body'].find("a", href=re.compile(r'jobPostingId%3D(\d{10})%26pivotType%3Dsim'))['href']).group(1), #re.search(re.compile(r'jobPostingId%3D(\d{10})%26pivotType%3Dsim', flags = re.DOTALL), x['body']).group(1),
                'position' : lambda x : x['subject'].split(" was viewed by ")[0],
                'company'  : lambda x : x['subject'].split(" was viewed by ")[1],
            },
            'rejected' : {
                # 'original_date_applied' : lambda x: dt.datetime.strptime(re.search(re.compile(r"Applied on (\w{3,9} \d{1,3}, \d{4})"), x['body']).group(1),'%B %d, %Y').strftime(CONSTS['date_format']),
                'position' : lambda x : x['subject'].split(" at ")[0],
                'company'  : lambda x : x['subject'].split(" at ")[1],
            },
        },
    },
    'TEMPLATE' : {
        'email': {
            'match' : lambda x: x,
            'scan' : {
                'date'        : lambda x: x,
                'update_type' : lambda x: x,
                'subject'     : lambda x: x,
                # 'title' : lambda x: x,
                # 'company' : lambda x: x,
            },
            'applied' : {
                'PostID'   : lambda x: x,
                'url'      : lambda x: x,
                'position' : lambda x: x,
                'company'  : lambda x: x,
            },
            'viewed' : {
                'PostID'   : lambda x: x,
                'position' : lambda x: x,
                'company'  : lambda x: x,
            },
            'rejected' : {
                'position' : lambda x: x,
                'company'  : lambda x: x,
            },
        },
    },
}

def cleaning_function_decorator(func):
    '''
    Wrapper that cleans all strings inside a list, dict, or string-like object.
    Converts numbers into float or int if applicable.
    Returns an object of the same format, but with cleaned strings.
    
    '''
    def wrapper(*args, **kwargs):
        results = func(*args,**kwargs)
        return clean_data(results)
    
    def clean_string(text):
        replace_chars = [('\n'," "), (re.compile(r"\s+")," ")]
        results = text
        for item in replace_chars:
            results = re.sub(item[0],item[1], results)
        results = results.strip()
        return results
    
    def clean_data(input_item):
        '''
        Recursive function that cleans strings inside objects.
        '''
        type_list = {"int":int, "str":str, "dict":dict, "list":list,}
        result_type = max([label if isinstance(input_item, type_example) else "" for (label,type_example) in type_list.items()]) #creates a list that checks the type of the input and returns a string representation of the type. Mostly using this so I can use match/case instead of an If/Else ladder.
        match result_type:
            case "str":
                print("string")
                if len(input_item)<256 and re.search(r"\d",input_item): #Typically not working with bigint, but will change as needed.
                    try: #This is going to hurt performance, but this will allow numerical strings to be relabeled as int or float
                        try:
                            return int(input_item.strip(","))
                        except:
                            return float(input_item.strip(","))
                    except:
                        return clean_string(input_item)
                return clean_string(input_item)
            case "dict":
                print("dictionary")
                input_item = {label:clean_data(item) for (label,item) in input_item.items()}
            case "list":
                print("list")
                return [clean_data(item) for item in input_item]
            case "int":
                print("int")
            case "float":
                print("float")
            case _:
                print(result_type)
        return input_item
    return wrapper

def handle_comprehension_errors(input_data, method_list):
    if "errors" not in input_data.keys():
        input_data |= {"errors":{}}
    for data_label in method_list: #loops through each data_label 
        try:
            input_data.update({data_label : method_list[data_label](input_data)}) #applies filter to captured data and inserts the key-value pair into input_data
        except Exception as e:
            input_data.update({data_label : None}) #If error, fills in value with null
            input_data["errors"].update({data_label:e}) #
    return input_data


def load_emails(directory):
    cached_emails = []
    file_list = os.listdir(directory)
    file_list = [file for file in file_list if ".eml" in file] #file not in read_files and 
    for item in file_list:
        path = directory+item
        with open(path, encoding="utf-8") as file:
            results = email.message_from_file(file)
            if results.is_multipart:
                # results = results.get_payload()
                body = results.get_payload()[1]
                print("Multipart")
            else:
                # results = results.get_payload()
                body = results.get_payload()
                print("Not Multipart")
            # print(results)
            body = re.sub(r"=$\n","", body.as_string(), flags=re.MULTILINE)
            headers = {part:re.sub(r"\n\s+"," ",value) for (part,value) in results.items()}
            html_contents = BeautifulSoup(body, "html5lib")
            data = { 
                'subject': headers['Subject'],
                'headers': headers,
                'body': html_contents,
                'from': headers['From']
            }
            
            cached_emails.append(data)
    return cached_emails

    
            # print(headers.keys())

def parse_email(data):
    input_method = 'email'

    # print(data['headers']['Subject'])
    sender = data['from'].split("@")[1].strip(">")
    platform = platform_identification[sender] if sender in platform_identification.keys() else "Unknown"
    # platform = 'LinkedIn'
    filters = platform_filters[platform][input_method]
    # filters = testing_platform_filters[platform][input_method]
    
    try:
        if filters['match'](data):
            email_data = {label:method(data) for (label,method) in filters['scan'].items()} #Initializes date, update_type, and subject
            email_data |= {label:method(email_data) for (label,method) in filters[data['update_type']].items()} #processes the above for more data
    except Exception as e:
        print(e, "!!!", data['subject'])
        print("Body: ", data['body'])
    return data


@cleaning_function_decorator
def parse_emails(input_data):
    results = [parse_email(item) for item in input_data]
    return results


    '''
    data = {
        "date:":headers['Received'].split(";")[1],
        "update_type": re.search(r'jobs?_appli\w{4,6}_([a-z]+)',headers['X-LinkedIn-Template']).group(1), 
        "JobID": re.search(r"(\d{10})",html_contents.find("a", href=re.compile(r"https://www.linkedin.com/comm/jobs/view/"))['href']).group(1),
        "Subject": headers['Subject'].replace(",","").replace(".","").replace("&", "and"), #re.search(r"Your? appli(cation|ed) (for|to) (.+)( at|was viewed by)(.+)",headers['Subject']), 
        "From":headers['From'],
        "To":headers['To'],
        # "Body":body,
    }
    # match data['update_type']:
    #     case 'applied':
    #         print(body)
    #         jobID = re.search( r'View job: https:(\S+&jobId=3D(\w+))' , html_contents, flags = re.DOTALL ).group(2)
    #     case 'viewed':
    #         jobID = re.search(r'jobPostingId%3D(\d{10})%26pivotType%3Dsim', html_contents, flags = re.DOTALL).group(1)
    #     case 'rejected':
    #         jobID = None
    #         # find jobID via match, perhaps. Maybe at later stage?
    #     case _:
    #         jobID = None
            
        

    # subject_filtering = re.search(re.compile("Your? appli(cation|ed) (for|to) (.+)( at|was viewed by)(.+)",flags=re.DOTALL),decode_mime_stuff(headers['Subject']))
    split_string = " was viewed by " if data['update_type']=="viewed" else " at "
    position, company = decode_mime_stuff(headers['Subject']).split(split_string)
    more_data = {
        "company": company, #subject_filtering.group(5),
        "position": position, #subject_filtering.group(3),
        "PostURL": 'https://www.linkedin.com/jobs/view/{}/'.format(data['JobID'])
    }
    # "JobID": re.search(r"(\d{10})",html_contents.find("a", href=re.compile(r"https://www.linkedin.com/comm/jobs/view/"))['href']).group(1),
    '''



In [ ]:
@cleaning_function_decorator
def test_function(input_dict):
    # print(input_dict)
    return input_dict

test_results = [{"Test":["123","420.69", "Dick123"],"Frog":"15", "Returns4":lambda x: "4","4":"12"}, {"Frog":"12"} ]

cleaned_results = test_function(test_results)
try:
    assert isinstance(cleaned_results[0]["Test"][1], float), "FAILED: Float not converted"
    assert isinstance(cleaned_results[0]["Test"][0], int), "FAILED: Int not converted"
    assert cleaned_results[0]['Returns4'](9999)=="4", "FAILED: Lambda converted"
    print("PASSED")
    print(cleaned_results)
except AssertionError as e:
    print(e)


In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import re
def getEmails(credpath, filters = None, maxRequests = None, query=None, nextPage=None, maxResults=None):
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    maxResults=500 if maxResults is None else maxResults
    cached_data = []
    duplicates = {}
    rejected_values = []
    # Variable creds will store the user access token.
    # If no valid token found, we will create one.
    creds = None
    pickle_path = '.\\dev\\models\\config\\token.pickle'
    pickle_path = re.search(r"(\.(\\\w+)+)\\\w+.json", credpath).group(1)+r"\\token.pickle"
    # The file token.pickle contains the user access token.
    # Check if it exists
    if os.path.exists(pickle_path):

        # Read the token from the file and store it in the variable creds
        with open(pickle_path, 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, ask the user to log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(f'{credpath}', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the access token in token.pickle file for the next run
        with open(pickle_path, 'wb') as token:
            pickle.dump(creds, token)

    # Connect to the Gmail API
    service = build('gmail', 'v1', credentials=creds)

    # request a list of all the messages
    if nextPage==None:
        result = service.users().messages().list(userId='me', maxResults=maxResults, q=query).execute()
    else:
        result = service.users().messages().list(userId='me', maxResults=maxResults, q=query, pageToken=nextPage).execute()
    
    # We can also pass maxResults to get any number of emails. Like this:
    # result = service.users().messages().list(maxResults=200, userId='me').execute()
    messages = result.get('messages')
    number_messages = len(messages)
    print("Number of messages for this round: ", number_messages)
    # messages is a list of dictionaries where each dictionary contains a message id.

            
    # iterate through all the messages
    for msg in messages:
        # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        # Use try-except to avoid any Errors
        # try:
        # Get value of 'payload' from dictionary 'txt'
        payload = txt['payload']
        headers = payload['headers']
        headers = {item['name']:item['value'] for item in headers}
        # Look for Subject and Sender Email in the headers
        filter_stuff = 'X-LinkedIn-Template' in headers.keys() #eventually call all email classification methods #TODO undo hardcoded linkedin
        target_values = ['jobs_applicant_applied', 'email_jobs_job_application_viewed_01', 'email_jobs_application_rejected_01']
        if filter_stuff: # Checks if 
            if headers['X-LinkedIn-Template'] not in target_values:
                rejected_values.append(headers['X-LinkedIn-Template'])
                continue
            print("FILTER RESULT", headers['X-LinkedIn-Template'])
            pass
        else:
            continue
        subject = headers['Subject']
        sender = headers['From']
        
        # The Body of the message is in Encrypted format. So, we have to decode it.
        # Get the data and decode it with base 64 decoder.
        parts = payload.get('parts')[0]
        data = parts['body']['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)

        # Now, the data obtained is in lxml. So, we will parse
        # it with BeautifulSoup library
        soup = BeautifulSoup(decoded_data , "lxml")
        body = soup.body()

        # Printing the subject, sender's email and message
        # wrapped_data = [headers,subject,sender,body]
        # print("Headers: ", headers)
        # print("Subject: ", subject)
        # print("From: ", sender)
        # print("Message: ", body)
        # print('\n')
        data = {
            "date_scanned":dt.now(),
            "subject":subject,
            "headers":headers,
            "body":body,
            "from":sender,
        }
        
        cached_data.append(data)
    # except Exception as e:
    # 	print(e,'"EXCEPTION')
    rejected_values = set(rejected_values)
    # print(cached_data)
    print("Duplicates", duplicates)
    print("Unfiltered Emails", rejected_values)
    if number_messages == maxResults: #capped query; need to rerun until all messages are parsed.
        print("Next round of parsing!")
        cached_data.extend(getEmails(credpath=credpath, filters=filters, query=query, nextPage=result['nextPageToken'], maxResults=maxResults)) # Creating a union between these sets. Allows for recursion.
    
    return cached_data

In [ ]:
credential_path = ".\\models\\config\\mika_wisener_gmail_credentials.json"
query=r'from:jobs-noreply@linkedin.com|jobs-listings@linkedin.com -"apply now|to" -"new job|jobs" -"don\'t forget"'
email_cache = getEmails(credpath=credential_path, query=query, maxResults=500)


In [ ]:
#Local Files
email_data = load_emails('.\\data\\test_input_files\\')

email_data = parse_emails(email_data)

In [ ]:
#Cloud
parsed_emails = parse_emails(email_cache)
app_history = pd.DataFrame(parsed_emails)
app_history

In [ ]:
# from .models.Parser import 

# parsed_features = self.scrape_html(input_data['url'])
# post_data = self.parser.scrape_posting(input_data['PostID'], parsing_library="html5lib", filter_path="selenium-client.post")
# company_data = self.parser.scrape_company_data(post_data)

# data.update(parsed_features)f
# data.update(post_data)



from dotenv import dotenv_values
from models.Parser import Parser
configuration = dotenv_values(".env")
p = Parser()
more_features = [p.add_features(item) for item in app_history]

In [ ]:
'''
SCRATCHWORK FOR LAMBDA FUNCTION CREATION

Creating a lambda function to convert a string containing a date into the desired date format.

Input: dictionary object containing email data
Output: formatted date string
'''


'''
EXAMPLE INPUT:
'''
dict_input =  {
    'subject': str(None),
    'body': BeautifulSoup("None", "html5lib"),
    'headers': {
        'Received' : 'by 2002:a05:6102:31b8:0:0:0:0 with SMTP id d24csp1274775vsh; Fri, 9 Sep 2022 11:37:46 -0700 (PDT)', #Location of date string
    },
}
match_pattern = '%d %b %Y %H:%M:%S %z' #Input date format
output_pattern = '%Y-%m-%d %H:%M:%S %Z' #Desired output date format


'''
FUNCTION DEFINITIONS:
Creating step by step lambdas to brainstorm the best order of the one-line solution.
The one-liner is made by collapsing each lambda with the one above

Ex:

lambda x : f(x)
lambda y : g(y)
lambda z : h(z)

becomes

lambda x : h(g(f(x)))
'''


#Individual functions to be chained together
input_method = lambda x: x['headers']['Received'].split(';')[1].split(",")[1].strip() #Locates and isolates the date string
cleaning_method = lambda x : re.sub(r"\s\(\w{3}\)","", x) #Removes time zone identifier. Since we already have the time-zone offset, we can avoid the parsing error where '%Z' doesn't match 'PDT'
convert_method = lambda x : dt.datetime.strptime(x, match_pattern) #converts string to datetime object
output_method = lambda x : x.strftime(output_pattern) #converts datetime object to desired string format.


#Parameterized version. Receives a clean string along with matching and desired date formats.
date_method = lambda string_input, match_pattern, output_pattern : dt.datetime.strptime(string_input, match_pattern).strftime(output_pattern)

#All the above as a one-line solution with parameterized formatting:
oneliner_method_with_parameters = lambda x : dt.datetime.strptime(re.sub(r"\s\(\w{3}\)","", x['headers']['Received'].split(';')[1].split(",")[1].strip()), match_pattern).strftime(output_pattern)
oneliner_method_without_parameters = lambda x : dt.datetime.strptime(re.sub(r"\s\(\w{3}\)","", x['headers']['Received'].split(';')[1].split(",")[1].strip()), '%d %b %Y %H:%M:%S %z').strftime('%Y-%m-%d %H:%M:%S %Z')


'''
OPERATIONS
'''

#Step by step operation:
input_test = input_method(dict_input) #Locates and isolates the date string
cleaning_test = cleaning_method(input_test) #Formats string for conversion
convert_test = convert_method(cleaning_test) #Converts string to a datetime object
output_test = output_method(convert_test) #Converts datetime object to desired format

#Variations on one-lining:
date_test = date_method(cleaning_test, match_pattern, output_pattern) #Converts a cleaned string to a datetime object according to the match_pattern, converts to desired output_pattern. Not used for this project, but has use elsewhere.
oneliner_test = oneliner_method_with_parameters(dict_input) #The one-line solution. Hard to read, but functional. This will be stored in our method dictionary.


'''
OUTPUTS
'''

print('input_test:\t',input_test)       #input_test:	 9 Sep 2022 11:37:46 -0700 (PDT)
print('cleaning_test:\t',cleaning_test) #cleaning_test:	 9 Sep 2022 11:37:46 -0700
print('convert_test:\t',convert_test)   #convert_test:	 2022-09-09 11:37:46-07:00
print('output_test:\t',output_test)     #output_test:	 2022-09-09 11:37:46 UTC-07:00

print('date_test:\t',date_test)         #date_test:      2022-09-09 11:37:46 UTC-07:00
print('oneliner_test:\t',oneliner_test) #oneliner_test:	 2022-09-09 11:37:46 UTC-07:00


In [ ]:



testing_platform_filters

input_data = None
if input_data("platform.email.match")
    _changelog = "platform.email.scan"
    _application_info = "platform.email.{}".format(_changelog["Update_Type"])
    if "LinkedIn":
        _new_results = [method_list for method_list in "platform.selenium-client".keys()]
        